<a href="https://colab.research.google.com/github/robitussin/CCADMACL_EXERCISES/blob/main/Exercise2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercise 2: Use Gradient Boost for Regression

Instructions:

- Use the Dataset File to train your model
- Use the Test File to generate your results
- Use the Sample Submission file to generate the same format
Submit your results to:
https://www.kaggle.com/competitions/playground-series-s4e12/overview



In [84]:
import pandas as pd
import seaborn as sns
from sklearn.impute import SimpleImputer
import numpy as np
from matplotlib import pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, train_test_split
import xgboost as xgb

## Dataset
Train, test and sample submission file can be found in this link
https://www.kaggle.com/competitions/playground-series-s4e12/data

## 1. Load the Data

In [101]:
# put your answer here
df = pd.read_csv('./exec2/train.csv')
X = df.drop(['id', 'Premium Amount'], axis=1)
y = df['Premium Amount']

## 2. Perform Data preprocessing

In [102]:
# put your answer here
cat_feat = X.select_dtypes(
    include=["object"]
).columns.tolist()

num_feat = X.select_dtypes(
    include=["float64", "int64"]
).columns.tolist()
# for col in cat_feat:
#    X[col] = X[col].astype('category')


In [103]:
X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.99, random_state=42
)
# dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
# dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [104]:
# params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
# n = 100
# model = xgb.train(
#    params=params,
#    dtrain=dtrain_reg,
#    num_boost_round=n,
# )

params = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.01,
}

## 3. Create a Pipeline

In [105]:
# put your answer here
# Preprocessing for categorical data

# numerical_transformer = StandardScaler()


numerical_transformer = SimpleImputer(strategy='mean', missing_values=np.nan)

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
    
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_feat),
        ('cat', categorical_transformer, cat_feat)
    ]
)
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ("regressor", GradientBoostingRegressor(**params)),
])



## 4. Train the Model

In [106]:
# put your answer here



cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5)



In [111]:
from sklearn.metrics import root_mean_squared_log_error

pipeline.fit(X_train, y_train)
y_predt = pipeline.predict(X_test)
report = root_mean_squared_log_error(y_test, y_predt)

In [108]:
report

1.1657945358691562

## 5. Evaluate the Model

In [114]:
# put your answer here

print(f"Mean Cross-Validation Accuracy: {cv_scores.mean():.4f}")
print("\nClassification Report:")
print(report)
y_predt

Mean Cross-Validation Accuracy: 0.0067

Classification Report:
1.1657945358689021


array([1113.39794792, 1113.39794792, 1119.93338645, ..., 1120.95491271,
       1063.24825928, 1120.95491271])

## Generate Submission File

Choose the model that has the best performance to generate a submission file.

In [ ]:
sf = pd.read_csv('./exec2/sample_submission.csv')
dt = pd.read_csv('./exec2/test.csv')
id = sf.pop('id')
y_pred = pipeline.predict(dt)

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'id': id,
    'Premium Amount': y_pred
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission_file_ex2.csv', index=False)
print("Submission file created: submission_file.csv")

Submission file created: submission_file.csv


In [118]:
id

0         1102.545
1         1102.545
2         1102.545
3         1102.545
4         1102.545
            ...   
799995    1102.545
799996    1102.545
799997    1102.545
799998    1102.545
799999    1102.545
Name: Premium Amount, Length: 800000, dtype: float64